In [1]:
import numpy as np
from random import randint
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

Новиков Г.К., 625.

Загрузка данных. Исходный датасет содержит информацию о ~18к футболистах и их характеристиках из игры FIFA 19 (думаю, не должен быть заезженным, к тому же, я люблю футбол). Рассматриваем только игроков с действующим контрактом (поле Wage не пустое и не равно 0) и только полевых игроков (Position!='GK'). Задача - предсказать потенциал игрока по его подробным характеристикам (дриблинг, скорость и т.п.), зарплате, цене, возрасту и общему рейтингу. Пробуем модели для регрессии.

In [2]:
f=open('football-players.txt',encoding='utf-8')
labels=[a.replace('\t\n','') for a in f.readline().split(',')]
data_labels=['Age','Overall','Value','Wage','International Reputation','Crossing','Finishing','HeadingAccuracy','ShortPassing','Volleys','Dribbling','Curve','FKAccuracy','LongPassing','BallControl','Acceleration','SprintSpeed','Agility','Reactions','Balance','ShotPower','Jumping','Stamina','Strength','LongShots','Aggression','Interceptions','Positioning','Vision','Penalties','Composure','Marking','StandingTackle','SlidingTackle']
target_label=['Potential']
target_data=[]
data_table=[]
for s in f:
    if 'GK' in s:
        continue
    attr=s.split(',')
    data=[]
    for i in range(len(labels)):
        if labels[i] not in data_labels:
            if labels[i] in target_label:
                target_data.append(int(attr[i]))
            else:
                continue
        else:
            if labels[i]=='Wage' or labels[i]=='Value':
                if attr[i]=='':
                    data=[]
                    break
                st=attr[i].replace('€','')
                if 'M' in st:
                    data.append(float(st.replace('M',''))*1000000)
                elif 'K' in st:
                    data.append(float(st.replace('K',''))*1000)
                else:
                    data=[]
                    break
            else:
                if (i>25):
                    if attr[i+1]=='':
                        data=[]
                        break
                    data.append(int(attr[i+1]))
                else:
                    if attr[i]=='':
                        data=[]
                        break
                    data.append(int(attr[i]))
    if data!=[]:
        data_table.append(data)
    if len(target_data)!=len(data_table):
        target_data.pop()
f.close()

Дальше поделим датасет на train и test и подготовим его. Test ~10% от всех данных.

In [3]:
train=[]
train_target=[]
test=[]
test_target=[]
for i in range(len(data_table)):
    if randint(0,9)==0:
        test.append(data_table[i])
        test_target.append(target_data[i])
    else:
        train.append(data_table[i])
        train_target.append(target_data[i])
print('Train: len=',len(train),', avg_target=',sum(train_target)/len(train_target),sep='')
print('Test: len=',len(test),', avg_target=',sum(test_target)/len(test_target),sep='')
train=np.array(train)
scaler=preprocessing.StandardScaler().fit(train)
train=scaler.transform(train)
train_target=np.array(train_target)
test=np.array(test)
test=scaler.transform(test)
test_target=np.array(test_target)
train_target=train_target/100
test_target=test_target/100

Train: len=14340, avg_target=71.52350069735007
Test: len=1578, avg_target=71.5278833967047


Теперь можно попробовать обучить и оценить линейную регрессию.

In [4]:
model1 = linear_model.TweedieRegressor(power=3,max_iter=1000)
model1.fit(train,train_target)
print('Percentage of deviance explained:',model1.score(test,test_target))
mse=mean_squared_error(test_target,model1.predict(test))
print('MSE:',mse)

Percentage of deviance explained: 0.6469789835863949
MSE: 0.0012178112574176914


Нормально вроде. MSE по логике надо бы на 100 умножить, чтобы вернуться к тем данным, которые были до нормализации, но даже так получается, что ошибка в ~0.1 на 100 пунктов рейтинга. Лучшие результаты получились с power=3 и power=0.

Дальше попробуем обучить деревья с разными параметрами.

In [5]:
tree1=DecisionTreeRegressor()
tree2=DecisionTreeRegressor(max_depth=5)
tree1.fit(train,train_target)
tree2.fit(train,train_target)
print('Tree1 determination of the prediction:',tree1.score(test,test_target))
mse=mean_squared_error(test_target,tree1.predict(test))
print('Tree1 MSE:',mse)
print('Tree2 determination of the prediction:',tree2.score(test,test_target))
mse=mean_squared_error(test_target,tree2.predict(test))
print('Tree2 MSE:',mse)

Tree1 determination of the prediction: 0.9380054042189935
Tree1 MSE: 0.00022313054499366287
Tree2 determination of the prediction: 0.8875076473973315
Tree2 MSE: 0.00040488174215247994


Не очень удивительно, что деревья оказываются лучше, так как количество возможных значений потенциала ограничено, и его не так сложно раскидать по листьям дерева, хотя я всё равно немного удивлён, что по характеристикам игрока можно настолько точно предсказать его потенциал, хоть я и ждал, что это возможно.

In [6]:
model2=svm.LinearSVR(max_iter=20000,tol=0.01)
model2.fit(train,train_target)
print('Coefficient of determination of the prediction:',model2.score(test,test_target))
mse=mean_squared_error(test_target,model2.predict(test))
print('MSE:',mse)

Coefficient of determination of the prediction: 0.834131182494441
MSE: 0.0005969939666710235


In [7]:
model3=KNeighborsRegressor()
model3.fit(train,train_target)
print('Coefficient of determination of the prediction:',model3.score(test,test_target))
mse=mean_squared_error(test_target,model3.predict(test))
print('MSE:',mse)

Coefficient of determination of the prediction: 0.7461936670879836
MSE: 0.0009134980988593156


In [8]:
model4=GradientBoostingRegressor(random_state=0)
model4.fit(train,train_target)
print('Coefficient of determination of the prediction:',model4.score(test,test_target))
mse=mean_squared_error(test_target,model4.predict(test))
print('MSE:',mse)

Coefficient of determination of the prediction: 0.9521634000926337
MSE: 0.00017217317854090444


А вот и новая лучшая модель, бустинг. Напоследок попробую нейронные сетки с разными параметрами.

In [9]:
nn1=MLPRegressor(max_iter=1000, solver='lbfgs',early_stopping=True)
nn1.fit(train,train_target)
print('NN1 Coefficient of determination of the prediction:',nn1.score(test,test_target))
mse=mean_squared_error(test_target,nn1.predict(test))
print('NN1 MSE:',mse)
nn2=MLPRegressor(max_iter=1000,batch_size=50)
nn2.fit(train,train_target)
print('NN2 Coefficient of determination of the prediction:',nn2.score(test,test_target))
mse=mean_squared_error(test_target,nn2.predict(test))
print('NN2 MSE:',mse)
nn3=MLPRegressor(max_iter=1000,hidden_layer_sizes=(100,100,100), solver='lbfgs',early_stopping=True)
nn3.fit(train,train_target)
print('NN3 Coefficient of determination of the prediction:',nn3.score(test,test_target))
mse=mean_squared_error(test_target,nn3.predict(test))
print('NN3 MSE:',mse)

NN1 Coefficient of determination of the prediction: 0.9076535839915977
NN1 MSE: 0.00033237261849329173
NN2 Coefficient of determination of the prediction: 0.8510575580221
NN2 MSE: 0.0005360726662145233
NN3 Coefficient of determination of the prediction: 0.9088017030577434
NN3 MSE: 0.00032824031583498153


Подведём итоги проделанной работы.
Были протестированы следующие модели (порядок такой же):
1. Линейная регрессия на обратном распределении Гаусса;
2. Дерево решений, одно без ограничения на глубину, одно с максимальной глубиной 5;
3. Регрессия на методе опорных векторов (SVM);
4. KNN регрессия с k=5;
5. Градиентный бустинг;
6. Нейронные сети, одна просто с lbfgs, одна с оптимизированным градиентным спуском (adam), одна с lbfgs и определённой вручную структурой.
Из всех моделей лучший результат показали градиентный бустинг, дерево без ограничения на глубину и обе нейронные сети с lbfgs.

В принципе из-за размера датасета (~16к строк) неудивительно, что lbfgs даёт лучший результат, ибо он ориентирован на небольшие датасеты.

Ну и от себя добавлю, пожалуй, пару слов. Во-первых, никак не доходили руки чисто самому что-то сделать с моделями ML, немного разобрался с sklearn (да и вообще юпитер поставил себе в кои-то веки). Во-вторых, лишний раз подтвердилось моё мнение, что сложнее подготовить датасет, чем обучать на нём модели, ибо почти все модели - копипаст (но не потому что я не понимаю, что делать, а потому что реально код одинаковый, только название модели разное), а подготовку датасета пришлось руками писать.